In [ ]:
%pip install --upgrade setuptools
%pip install --upgrade gradio
%pip install ipywidgets
%pip install pandas
%pip install tabulate
%pip install python-dotenv

In [ ]:
%pip install -e ../

In [ ]:
import json
import setuptools

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from dotenv import dotenv_values
from typechat import Failure, TypeChatJsonTranslator, TypeChatValidator, create_language_model
from examples.calendar import schema as calendar

In [ ]:
model = create_language_model(dotenv_values())
validator = TypeChatValidator(calendar.CalendarActions)
translator = TypeChatJsonTranslator(model, validator, calendar.CalendarActions)

In [ ]:
import pandas

async def get_translation(message, history):
    result = await translator.translate(message)
    if isinstance(result, Failure):
        return f"Translation Failed ❌ \n Context: {result.message}"
    else:
        df = pandas.DataFrame.from_dict(result["actions"])
        return f"Translation Succeeded! ✅\n Table View \n ``` {df.fillna('').to_markdown(tablefmt='grid')} \n ```  \n"

def get_examples():
    example_prompts = []
    with open('../examples/calendar/input.txt') as prompts_file:
        for line in prompts_file:
            example_prompts.append(line)
    return example_prompts

In [ ]:
import gradio as gr

gr.ChatInterface(get_translation, title="📅 Calendar", examples=get_examples()).launch()
